In [1]:
#%%
print('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting')

from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
#%%
seed = 42
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)


/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting


In [ ]:
def data_split(aa_array, encoded_mic):
    # Encode the target variable
    # Perform stratified train-test split
    train_data, test_data, train_target, test_target = train_test_split(
        aa_array,
        encoded_mic,
        test_size=0.1,  # 10% for testing
        stratify=encoded_mic,  # Ensures the proportion of each class is preserved
        random_state=42  # For reproducibility
    )
    return train_data, test_data, train_target, test_target
def is_within_doubling_dilution(pred, target, target_min, target_max):
    _ = np.arange(target_min-1, target_max+2, 1)
    index = [i for i, x in enumerate(_) if x == target][0]
    return (_[index-1] <= pred <= _[index+1])
def data_prep_(cryptic, gene_list, dr_list):

    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
    overlap = overlap['sample_id'].unique()

    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    variants = variants[variants['SNP'].isin(crypticSNPnames)]
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list

    aa = []
    dr = []
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    # np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
            if dr_list[0] in variants[variants['sample_id']==x]['drugs'].unique():
                dr.append(1)
            else:
                dr.append(0)
        else:
            # aa.append([0]*len(all_snp))
            pass
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    return aa_array, dr

## rifampicin

In [ ]:

df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
gene_list = ['rpoB']
dr_list = ['rifampicin']
df_emb = df[df['RIF_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = 'RIF_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.125'

variants = pd.read_csv('../variants_full.csv')
variants = variants[variants['gene'].isin(gene_list)]
crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)

        
df_emb['RIF_MIC'] = df_emb['RIF_MIC'].astype('float') 

cryptic = df_emb
aa_array, drs = data_prep_(cryptic, gene_list,dr_list)

with open('all_sample_snps_cryptic_rif.npy', 'wb') as f:
    np.save(f, aa_array)
with open('all_sample_drs_cryptic_rif.npy', 'wb') as f:
    np.save(f, np.array(drs))
